# ゼロから作る Deep Learning 4 強化学習編 勉強ノート 第1章 〜バンディット問題〜

ここでは、強化学習において最も基本となる「<font color="red">**バンディット問題**</font>」について扱う。

## 機械学習の区分

機械学習で使われる手法は、解きたい問題の構造によって分けられる。  
代表的な区分は以下の3つである。

- <font color="red">**教師あり学習**</font>
- <font color="red">**教師なし学習**</font>
- <font color="red">**強化学習**</font>

### 教師あり学習

教師あり学習(Supervised Learning)では、入力と出力のペアデータが与えられる。  
入力するデータから、何らかのアルゴリズムのもとで出力したいデータを得られるように学習し、予測する。  
例えば、「手書き数字の画像」と「正解ラベル」が与えられたとすると、画像の特徴からペアになる正解ラベルと結びつくように学習させることで、未知の画像を使っても同様にラベルを予測することができる。

教師あり学習の中にも、大きく分けて2種類のタスクがある。

- <font color="red">**分類**</font>
- <font color="red">**回帰**</font>

先述の画像からラベルを予測する問題は、「分類問題」に相当する。  
分類問題では、出力の値が離散値、またはテキストなどになる。  
株価・為替レートや1日の平均気温など、連続した値を取るデータを予測する問題は、「回帰問題」に相当する。  
回帰問題では、出力の値が連続値になる。

教師あり学習の中で、入力として与えるデータのことを「<font color="red">**説明変数**</font>」といい、出力として与えるデータのことを「<font color="red">**目的変数**</font>」という。  
教師あり学習(主に分類問題)の場合、その多くは目的変数を人の手で用意しなければいけないため、データの規模が大きくなるにつれてコストがかかる。

### 教師なし学習

教師なし学習(Unsupervised Learning)では、「教師」が存在しない。  
すなわち、予測してほしい正解データが手元に無く、ただのデータだけがある状態で何かしらの構造やパターンを見つけるものである。

例えば<font color="red">**クラスタリング**</font>では、ルールが定められていないテーブルデータから、指定した数のグループにそれぞれ近いと判断されたデータ同士が固まるように分けるといったことをする。  
また、<font color="red">**主成分分析**</font>では、用意されたデータの変数が冗長であるとき(あまり関係性がなさそうな変数や、その変数を包含するような変数が他にあった場合)、必要な数だけ重要な変数を順に選択し、次元の削減を行う。

教師なし学習は正解データが不要なため、ビッグデータが与えられても比較的容易に準備することができる。

### 強化学習

強化学習(Reinforcement Learning)では、「<font color="red">**エージェント**</font>」と「<font color="red">**環境**</font>」が相互にやり取りをすることによって学習を進める。

![強化学習](https://www.skillupai.com/wp-content/uploads/2022/01/reinforcement-learning_01.png.webp)

(出典: https://www.skillupai.com/blog/tech/reinforcement-learning/)

エージェントとは、ロボットのような行動を起こすもののことである。  
このエージェントは何らかの環境に置かれており、その環境の「状態」を観測し、それに基づいて「行動」を起こす。  
その結果として環境の状態が変化し、エージェントは環境から「報酬」を受け取ると同時に新しい「状態」を観測する。  
強化学習の目的は、エージェントが得る報酬の総和を最大にする行動パターンを身につけることである。

![DQN](https://pytorch.org/tutorials/_images/cartpole.gif)

(出典: https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html)

強化学習では、環境からのフィードバックとして報酬を受け取るが、これは教師あり学習で言う正解データとは性質が異なる。  
報酬は行動に対してのフィードバックであり、その報酬からは実際に行った行動が最適かどうかは判断できない。  
一方、教師あり学習では正解データがあり、これを強化学習の文脈で言えば、どんな行動を取っても最適な行動はこれだと提示してくれる教師がいる。  
つまり強化学習では、自分で試行錯誤してデータを集めて、その中から良い行動の作法を学習しなければいけないということである。

この章では、強化学習の中でも最もシンプルな「<font color="red">**バンディット問題**</font>」を取り扱う。

## バンディット問題とは

バンディット問題(Bandit Problem)を扱う上で、1本のレバーが付いたスロットマシンが複数台並べてあることをイメージする。  
各スロットマシンは特性が異なり、当たり外れが違うとする。  
初めプレイヤーは、どのマシンが当たりやすいかという情報を持っていないとし、実際にプレイして試行錯誤しながらどのマシンが良いかを見極めていく必要がある。  
更に、予め決められた回数の中で得られるコインの枚数をできるだけ多くすることを目標とする。

今回、スロットマシンは「環境(Environment)」であり、プレイヤーが「エージェント(Agent)」である。  
この2つの登場人物が相互に「やり取り」するというのが強化学習の枠組みである。  
プレイヤーが複数あるスロットマシンから1台選び、プレイすることが「行動(Action)」である。  
そして、その結果としてプレイヤーが受け取るコインが「報酬(Reward)」である。  
「状態(State)」は一般的に環境の内部に備わっているものとする。  
ここでは状態の変化を考える必要がない。

### 良いスロットマシンを考える

スロットマシンには「ランダム性」がある。  
ランダム性とは、スロットマシンをプレイして得られるコインの枚数(報酬)が毎回変わるということを言う。  
例えば以下の2種類のスロットマシン $a, b$ を考える。

| もらえる<br>コインの枚数 | 0 | 1 | 5 | 10 |
| :--: | :--: | :--: | :--: | :--: |
| $a$ | 0.70 | 0.15 | 0.12 | 0.03 |
| $b$ | 0.50 | 0.40 | 0.09 | 0.01 |

この表は<font color="red">**確率分布表**</font>と呼ばれる。  
通常プレイヤーはこのような確率分布表が分かっていないが、ここでは敢えて分かっているとする。  
このとき、プレイヤーはどちらのスロットマシンを選ぶのが良いだろうか。

確率分布表を使って、<font color="red">**期待値**</font>(Expectation Value)を算出する。

$$
a: 0 \times 0.70 + 1 \times 0.15 + 5 \times 0.12 + 10 \times 0.03 = 1.05 \\
b: 0 \times 0.50 + 1 \times 0.40 + 5 \times 0.09 + 10 \times 0.01 = 0.95
$$

以上より、 $a$ を選んだ方が1回辺りの期待値は高いので良いと言える。

### 数式を使った表記

まず、報酬を $R$ と表すとする。  
更に、エージェントは連続して行動を起こすので、 $t$ 回目に得られる報酬を明示的に表す場合は $R_t$ と書くことにする。  
ここで、先ほどの例を使うのであれば、 $R \in \{0, 1, 5, 10\}$ である。  
このように、取る値が確率的に決まる変数のことを<font color="red">**確率変数**</font>(Random Variable)と言う。

続いて、エージェントの行う行動を $A$ とする。  
エージェントの行動は $a$ または $b$ のスロットマシンでプレイすることなので、 $A \in \{a, b\}$ と表せる。

次に、期待値は $\mathbb{E}$ で表す。  
例えば、報酬 $R$ の期待値は $\mathbb{E}[R]$ のように書く。  
条件付き期待値は $\mathbb{E}[R|A]$ と書く。  
更に、エージェントが例えば $a$ を選んだとすると、 $\mathbb{E}[R|A = a]$ または単に $\mathbb{E}[R|a]$ と記述する。

報酬の期待値は<font color="red">**行動価値**</font>(Action Value)と呼ばれ、強化学習の分野では $Q$ や $q$ という記号をあてる。  
ここでは

$$
q(A) = \mathbb{E}[R|A]
$$

とする。  
一般的には、小文字を使うときは「真の行動価値」を表し、大文字を使うときは「推定した行動価値」を表す。

## バンディットアルゴリズム

ここまでの話を整理すると次のようになる。

- もし各スロットマシンの価値(報酬の期待値)がわかれば、プレイヤーは最も良いスロットマシンを選ぶことができる
- しかし各スロットマシンの価値はプレイヤーにはわからない
- よってプレイヤーには、各スロットマシンの価値を(できるだけ精度良く)推定することが求められる

上記の点を満たし、得られるコインの総和を高める方策が求められる。

### 価値の推定

| スロットマシン | 1回目 | 2回目 | 3回目 |
| :--: | :--: | :--: | :--: |
| $a$ | 0 | 1 | 5 |
| $b$ | 1 | 0 | 0 |

この表は、各マシンで3回ずつプレイして得た報酬をまとめたものであるとする。  
1回目の結果だけを見て、 $b$ のマシンの方が良いと判断するのはあまり良くない。  
実際3回目までやると $a$ のマシンの方が良い結果が得られている。  
ここで、それぞれのマシンの価値の推定値 $Q(a), Q(b)$ は次のように計算できる。

\begin{eqnarray*}
Q(a) &=& \frac{0 + 1 + 5}{3} = 2 \\
Q(b) &=& \frac{1 + 0 + 0}{3} = 0.33 \cdots
\end{eqnarray*}

これを一般化すると、次のようになる。  
試行回数を $T$ 回とすると、

$$
Q_T(A) = \mathbb{E}[R|A] = \frac{1}{T} \sum_{t=1}^T R_t
$$

であり、この $T$ を $\infty$ にとばすと、

$$
\lim_{T\to \infty} Q_T(A) = q(A)
$$

となる。  
すなわち、試行回数を重ねるにつれて真の確率分布に近づく。

### 期待値を求める(*コーディング*)

ここでは、 $T = 10$ として報酬を得るたびに推定値を求めるコードを作成する。

In [ ]:
# ライブラリのインポート
import numpy as np

In [ ]:
# 乱数シードの固定
np.random.seed(8192)

# 報酬を記録するための空リストを用意
rewards = []

# i 回目の報酬の推定結果を出力
for i in range(1, 11):
    reward = np.random.rand()
    rewards.append(reward)
    q = sum(rewards) / i
    print(q)

上記の実装では正しく報酬の期待値を算出することができるが、改善点もある。  
試行回数 $T$ が増えるにつれて `rewards` の要素が増えてしまう。  
また、 $T$ 個分の和を求めるための計算量(`sum(rewards)`)も同様に増える。

より良い実装をするために、数式を使って説明をする。  
まずは $T - 1$ 回目の時点での行動価値の推定値である $Q_{T-1}$ に注目する。

$$
Q_{T-1} = \frac{1}{T-1} \sum_{t=1}^{T-1} R_t
$$

ここで、両辺に $T-1$ をかけると

$$
\sum_{t=1}^{T-1} R_t = (T-1)Q_{T-1}
$$

が得られる。  
$Q_T$ は、

\begin{eqnarray*}
Q_T &=& \frac{1}{T} \sum_{t=1}^{T} R_t \\
&=& \frac{1}{T} \left( \left( \sum_{t=1}^{T-1} R_t \right) + R_T \right) \\
&=& \frac{1}{T} \left( (T-1)Q_{T-1} + R_T \right) \\
&=& \left( 1 - \frac{1}{T} \right) Q_{T-1} + \frac{1}{T}R_T
\end{eqnarray*}

と表せる。  
ポイントは、 $Q_T$ と $Q_{T-1}$ の関係が導かれたことにより、 $Q_T$ を求めるのに $Q_{T-1}$ と $R_T$ 、そして $T$ の3つの値があれば良いことがわかる。  
よって今までで得られた全ての報酬を毎回使わなくても使わなくても良いという事になる。

\begin{eqnarray*}
Q_T &=& \left( 1 - \frac{1}{T} \right) Q_{T-1} + \frac{1}{T}R_T \\
&=& Q_{T-1} + \frac{1}{T}(R_T - Q_{T-1})
\end{eqnarray*}

$Q_{T-1}$ から $Q_T$ に更新するには、 $\frac{1}{T}(R_T - Q_{T-1})$ を足してあげれば良いということが分かった。  
ここで、 $\frac{1}{T}$ は更新する量を調整するので「学習率」としての役割を担う。

以上を踏まえて再度実装し直してみる。

In [ ]:
# 推定値の初期化
q = 0

# 改善したアルゴリズムで　i 回目の報酬の推定結果を出力
for i in range(1, 11):
    reward = np.random.rand()
    q = q + (reward - q) / i # q += (reward - q) / i
    print(q)

### プレイヤーの戦略

プレイヤーに求められるのは次の2点になる。

- これまで実際にプレイした結果を利用して、最善と思われるスロットマシンをプレイすること(<font color="red">**greedy な行動**</font>)
- スロットマシンの価値を精度良く推定するために、様々なスロットマシンを試すこと

greedy な行動をすることで、これまでの経験に基づいて最善の行動を選択できるが、先の例では1回目の結果からは $b$ のスロットマシンが選ばれ続けることになる。  
この選択を「<font color="red">**活用**</font>(Exploitation)」とも言う。  
一方で、より良い選択を見逃さないように「greedy でない行動」を試す必要があり、これを「<font color="red">**探索**</font>(Exploration)」と言う。  
活用と探索はトレードオフの関係にあり、強化学習のアルゴリズムではこのバランスをいかに取るかが問題になる。  
このバランスを取る方法として最も基本的で応用の利くアルゴリズムに「<font color="red">**ε-greedy 法**</font>(イプシロン-グリーディー法)」がある。

## ε-greedy 法の実装

話を単純にするために、スロットマシンが返すコインの枚数を0枚か1枚のどちらかに限られる場合を考えることにする。  
各スロットマシンには勝率が設定されているとし、例えば勝率が0.6であるマシンは60%の確率で1を返し、40%の確率で0を返すとする。  
勝率はそのままスロットマシンの価値になる。  
10台のスロットマシンを考え、プレイヤーはそれぞれのスロットマシンに設定された勝率を知らないものとする。  
そのため、実際にプレイした経験をもとに勝率の高いスロットマシンを探す必要がある。

### スロットマシンの実装(*コーディング*)

スロットマシンを `Bandit` クラスとして実装する。  
このクラスの中に10台のマシンが存在するように実装する。

In [ ]:
# ライブラリのインポート
import numpy as np

In [ ]:
# Bandit クラスの実装
class Bandit:
    def __init__(self, arms=10):
        self.rates = np.random.rand(arms)

    def play(self, arm):
        rate = self.rates[arm]
        if rate > np.random.rand():
            return 1
        else:
            return 0

`play()` メソッドでは、何番目のスロットマシンを使うかを表す `arm` を引数に指定する。  
`__init__()` メソッドで指定したそれぞれの勝率を閾値として、生成した乱数がそれを下回れば1を、上回ればを返す。

In [ ]:
# インスタンス生成
bandit = Bandit()

# 0番目のスロットマシンを3回プレイ
for i in range(3):
    print(bandit.play(0))

### エージェントの実装(*コーディング*)

まずは今作った `Bandit` クラスを使って0番目のスロットマシンを10回プレイしたときのその価値の推定をしてみる。

In [ ]:
# インスタンス生成
bandit = Bandit()

# 推定値の初期化
q = 0

# インクリメンタルな実装で推定結果を出力
for i in range(1, 11):
    reward = bandit.play(0)
    q += (reward - q) / i
    print(q)

これを拡張し、10台のスロットマシンそれぞれの価値の推定値を求める。

In [ ]:
# インスタンス生成
bandit = Bandit()

# 対応するスロットマシンの価値の推定値を格納する
qs = np.zeros(10)

# 対応するスロットマシンをプレイした回数を格納する
ts = np.zeros(10)

# インクリメンタルな実装で1000回プレイした推定結果を出力
for i in range(1000):
    action = np.random.randint(0, 10)
    reward = bandit.play(action)

    ts[action] += 1
    qs[action] += (reward - qs[action]) / ts[action]

print(qs)

これで各スロットマシンの価値が推定できた。  
これを基に `Agent` クラスを実装する。  
この `Agent` クラスは、ε-greedy法によって行動を選択するアルゴリズムを実装している。

In [ ]:
# Agent クラスの実装
class Agent:
    def __init__(self, eps, action_size=10):
        self.eps = eps
        self.qs = np.zeros(action_size)
        self.ts = np.zeros(action_size)

    def update(self, action, reward):
        self.ts[action] += 1
        self.qs[action] += (reward - self.qs[action]) / self.ts[action]

    def get_action(self):
        if np.random.rand() < self.eps:
            return np.random.randint(0, len(self.qs))
        return np.argmax(self.qs)

引数の `eps` にはエージェントがランダムな行動を行う確率を指定する。  
`update()` メソッドでは、スロットマシンの価値を推定し、 `get_action()` メソッドでは、ε-greedy法によって行動を選択する。

### 動かしてみる(*コーディング*)

これまでに実装してきた `Bandit` クラスと `Agent` クラスを使って動かしてみる。

In [ ]:
# ライブラリのインポート
import matplotlib.pyplot as plt

In [ ]:
# グラフを描画する関数を定義
def plotter(
        data, xlabel: str, ylabel: str,
        figsize_x: int=18, figsize_y: int=12
    ):
    plt.figure(figsize=(figsize_x, figsize_y))
    plt.grid()

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(data)

    plt.show()

In [ ]:
# ハイパーパラメータの設定
steps = 1000
eps = 0.1

# インスタンスの生成と各種データの初期化
bandit = Bandit()
agent = Agent(eps)
total_reward = 0
total_rewards = []
rates = []

In [ ]:
# インクリメンタルな実装で1000回プレイした推定結果を出力
for step in range(steps):
    action = agent.get_action()
    reward = bandit.play(action)
    agent.update(action, reward)
    total_reward += reward

    total_rewards.append(total_reward)
    rates.append(total_reward / (step + 1))

print(total_reward)

In [ ]:
# 報酬の総量をグラフに描画
plotter(total_rewards, xlabel="Steps", ylabel="Total reward")

In [ ]:
# 勝率をグラフに描画
plotter(rates, xlabel="Steps", ylabel="Rates")

### 平均的な性質(*コーディング*)

先ほどの実装では、乱数のシードを固定しなければ実行ごとに毎回結果が変わってしまう。  
試しに10回実行した結果をグラフに描画してみる。

In [ ]:
# 10回分に拡張した描画関数
def plotter_ten(
        data, xlabel: str, ylabel: str,
        figsize_x: int=18, figsize_y: int=12
    ):
    plt.figure(figsize=(figsize_x, figsize_y))
    plt.grid()

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    for i in range(10):
        plt.plot(data[i])

    plt.show()

In [ ]:
# ハイパーパラメータの設定
steps = 1000
eps = 0.1

# 10回分のデータを格納する
rate_data = []

# 10回繰り返す
for i in range(10):
    bandit = Bandit()
    agent = Agent(eps)
    total_reward = 0
    total_rewards = []
    rates = []

    for step in range(steps):
        action = agent.get_action()
        reward = bandit.play(action)
        agent.update(action, reward)
        total_reward += reward

        total_rewards.append(total_reward)
        rates.append(total_reward / (step + 1))

    rate_data.append(rates)

# グラフの描画
plotter_ten(rate_data, xlabel="Steps", ylabel="Rates")

強化学習のアルゴリズムを比較するとき、ランダム性があるため1回の実験結果を報告することにあまり意味はない。  
同じ実験を繰り返し行い、その平均を見ることに意味がある。  
次は、200回繰り返した実験の平均を考えてみる。

In [ ]:
# ハイパーパラメータの設定
runs = 200
steps = 1000
eps = 0.1
all_rates = np.zeros((runs, steps))

# 200回の実験
for run in range(runs):
    bandit = Bandit()
    agent = Agent(eps)
    total_reward = 0
    rates = []

    for step in range(steps):
        action = agent.get_action()
        reward = bandit.play(action)
        agent.update(action, reward)
        total_reward += reward
        rates.append(total_reward / (step + 1))

    all_rates[run] = rates

# 平均の算出
avg_rates = np.average(all_rates, axis=0)

# 勝率をグラフに描画
plotter(avg_rates, xlabel="Steps", ylabel="Rates")

## 非定常問題

ここまでは、定常問題として考えられる。  
定常問題とは、報酬の確率分布が定常である問題のことで、スロットマシンに設定した勝率が固定されていた。

```python
class Bandit:
    def __init__(self, arms=10):
        self.rates = np.random.rand(arms)

    def play(self, arm):
        rate = self.rates[arm]
        if rate > np.random.rand():
            return 1
        else:
            return 0
```

このコードでは、 `self.rates` は初期化された後は変化しない。  
これをプレイするごとに少しずつ変化させる問題を考える。

### 非定常問題を解くにあたって

標本平均は次の式で表せた。

\begin{eqnarray*}
Q_T &=& \frac{R_1 + R_2 \cdots + R_T}{T} \\
&=& \frac{1}{T}R_1 + \frac{1}{T}R_2 + \cdots + \frac{1}{T}R_T
\end{eqnarray*}

ここで、全ての報酬 $R_t$ に対して $\frac{1}{T}$ が掛かっている。  
この $\frac{1}{T}$ を各報酬に対する「重み」と見做す。  
全ての報酬は同じ重みなので、新しく得た報酬も遥か前に得た報酬も均等に扱ってしまっている。  
非定常問題では環境が時間と共に変化するため、過去のデータの重要性は時間と共に低くなるべきである。

上記の $Q_T$ は次のようにも表せた。

$$Q_T = Q_{T-1} + \frac{1}{T}(R_T - Q_{T-1})$$

ここの $\frac{1}{T}$ を $\alpha \space (0 < \alpha < 1)$ という固定値に変更すると、

\begin{eqnarray*}
Q_T &=& Q_{T-1} + \alpha (R_T - Q_{T-1}) \\
&=& \alpha R_T + Q_{T-1} - \alpha Q_{T-1} \\
&=& \alpha R_T + (1 - \alpha)Q_{T-1}
\end{eqnarray*}

ここで、 $T-1$ の場合を考えると、

$$Q_{T-1} = \alpha R_{T-1} + (1 - \alpha)Q_{T-2}$$

であるので上記式に代入し、再帰的に展開すると

\begin{eqnarray*}
Q_T &=& \alpha R_T + (1 - \alpha)Q_{T-1} \\
&=& \alpha R_T + (1 - \alpha)(\alpha R_{T-1} + (1 - \alpha)Q_{T-2}) \\
&=& \alpha R_T + \alpha (1 - \alpha)R_{T-1} + (1 - \alpha)^2 Q_{T-2} \\
&=& \alpha R_T + \alpha (1 - \alpha)R_{T-1} + \alpha (1 - \alpha)^2 R_{T-2} + (1 - \alpha)^3 Q_{T-3} \\
&=& \alpha R_T + \alpha (1 - \alpha)R_{T-1} + \cdots + \alpha (1 - \alpha)^{T-1} R_{1} + (1 - \alpha)^T Q_{0}
\end{eqnarray*}

となる。  
各報酬に対する重みが指数関数的に減少していることが確認できる。  
これらは<font color="red">**指数移動平均**</font>(Exponential Moving Average)や<font color="red">**指数加重移動平均**</font>(Exponential Weighted Moving Average)と言う。

ここで注意すべき点は、 $Q_T$ を求める上で、 $Q_0$ が使われていることである。  
$Q_0$ は行動価値の初期値であり、自分で設定する値である。  
故にこの設定した値によっては学習結果にバイアスが生じてしまうことがある。

### 非定常問題を解く(*コーディング*)

まずは `Bandit` クラスの `self.rates` を変化させられる `NonStatBandit` クラスを実装する。

In [ ]:
# NonStatBandit クラスの実装
class NonStatBandit:
    def __init__(self, arms=10):
        self.arms = arms
        self.rates = np.random.rand(arms)

    def play(self, arm):
        rate = self.rates[arm]
        self.rates += 0.1 * np.random.randn(self.arms)
        if rate > np.random.rand():
            return 1
        else:
            return 0

続いて、 `Agent` クラスに変更を加え、推定値を固定値 $\alpha$ で更新する `AlphaAgent` クラスを実装する。

In [ ]:
# AlphaAgent クラスの実装
class AlphaAgent:
    def __init__(self, eps, alpha, actions=10):
        self.eps = eps
        self.qs = np.zeros(actions)
        self.alpha = alpha

    def update(self, action, reward):
        self.qs[action] += (reward - self.qs[action]) * self.alpha

    def get_action(self):
        if np.random.rand() < self.eps:
            return np.random.randint(0, len(self.qs))
        return np.argmax(self.qs)

これらを使って定常問題と同様に実験をしてみる。

In [ ]:
# ハイパーパラメータの設定
runs = 200
steps = 1000
eps = 0.1
alpha = 0.8
all_rates = np.zeros((runs, steps))

# 200回の実験
for run in range(runs):
    bandit = NonStatBandit()
    agent = AlphaAgent(eps, alpha)
    total_reward = 0
    rates = []

    for step in range(steps):
        action = agent.get_action()
        reward = bandit.play(action)
        agent.update(action, reward)
        total_reward += reward
        rates.append(total_reward / (step + 1))

    all_rates[run] = rates

# 平均の算出
avg_rates = np.average(all_rates, axis=0)

# 勝率をグラフに描画
plotter(avg_rates, xlabel="Steps", ylabel="Rates")